# FCPM Workflow 2: Reconstruction from Experimental Data

This notebook demonstrates how to reconstruct a director field from **experimental FCPM data** where you don't have a ground truth for comparison.

**Workflow:**
1. Load FCPM intensity images
2. Preprocess (crop, filter, normalize)
3. Reconstruct director field
4. Assess quality using self-consistency metrics
5. Visualize and save results

**Use case**: Processing real microscopy data, analyzing unknown liquid crystal structures.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import fcpm

print(f"FCPM Library version: {fcpm.__version__}")

FCPM Library version: 1.1.0


## Option A: Quick Workflow (One Function Call)

The simplest approach using `run_reconstruction()`:

In [2]:
# For this demo, we'll create synthetic "experimental" data
# In practice, replace this with your actual FCPM data file

# Create simulated experimental data
director = fcpm.create_cholesteric_director(shape=(48, 48, 24), pitch=6.0)
I_fcpm_exp = fcpm.simulate_fcpm(director)
I_fcpm_exp = fcpm.add_fcpm_realistic_noise(I_fcpm_exp, noise_model='mixed', gaussian_sigma=0.05)

# Save it as "experimental" data
Path('../output').mkdir(exist_ok=True)
fcpm.save_fcpm_npz(I_fcpm_exp, '../output/experimental_fcpm.npz')
print("Created simulated experimental data: ../output/experimental_fcpm.npz")

Created simulated experimental data: ../output/experimental_fcpm.npz


In [3]:
# Configure reconstruction workflow
config = fcpm.WorkflowConfig(
    crop_size=None,         # No cropping (data already small)
    filter_sigma=0.5,       # Light Gaussian smoothing
    fix_signs=True,         # Resolve sign ambiguity
    save_plots=True,
    save_data=True,
    verbose=True,
)

# Run reconstruction
results = fcpm.run_reconstruction(
    fcpm_source='../output/experimental_fcpm.npz',
    output_dir='../output/notebook_workflow2',
    config=config
)

FCPM Reconstruction Workflow

[1/4] Loading FCPM data...
      Shape: (48, 48, 24)
      Angles: 4

[2/4] No cropping
      Applied Gaussian filter (sigma=0.5)

[3/4] Reconstructing from FCPM...
Combined Sign Optimization

Phase 1: Chain propagation...
  After chain propagation: energy = 57073.92

Phase 2: Local flip refinement...
Initial gradient energy: 57073.92
  Iter 0: flipped 0 voxels, energy = 57073.92
Converged at iteration 0

[4/4] Computing metrics...

Results Summary
  Reconstructed shape: (48, 48, 24)
  Gradient energy:     57073.92
  Intensity RMSE:      1.54e-01

Saving results to: ../output/notebook_workflow2
  Saved data files to: ../output/notebook_workflow2/data
  Saved plots to: ../output/notebook_workflow2/plots
  Saved summary to: ../output/notebook_workflow2/summary.json

Workflow complete!


In [4]:
# Check results
print("\n=== Results Summary ===")
print(f"Reconstructed shape: {results.director_recon.shape}")
print(f"Gradient energy: {results.metrics['gradient_energy']:.2f}")
print(f"Intensity RMSE (self-consistency): {results.metrics['intensity_rmse_mean']:.4f}")
print(f"Ground truth available: {results.director_gt is not None}")


=== Results Summary ===
Reconstructed shape: (48, 48, 24)
Gradient energy: 57073.92
Intensity RMSE (self-consistency): 0.1543
Ground truth available: False


## Option B: Step-by-Step Workflow

For more control over each processing step:

### Step 1: Load FCPM Data

The library supports multiple formats:
- NPZ files with angle keys
- TIFF stacks
- MATLAB .mat files
- NumPy arrays directly

In [5]:
# Load FCPM data (auto-detects format)
I_fcpm = fcpm.load_fcpm('../output/experimental_fcpm.npz')

print("Loaded FCPM data:")
print(f"  Number of angles: {len(I_fcpm)}")
for angle, intensity in I_fcpm.items():
    print(f"  Angle {np.degrees(angle):.1f} deg: shape {intensity.shape}, "
          f"range [{intensity.min():.3f}, {intensity.max():.3f}]")

Loaded FCPM data:
  Number of angles: 4
  Angle 0.0 deg: shape (48, 48, 24), range [0.000, 1.216]
  Angle 45.0 deg: shape (48, 48, 24), range [0.000, 1.210]
  Angle 90.0 deg: shape (48, 48, 24), range [0.000, 1.218]
  Angle 135.0 deg: shape (48, 48, 24), range [0.000, 1.202]


In [6]:
# Visualize raw FCPM data
z_mid = next(iter(I_fcpm.values())).shape[2] // 2
fig = fcpm.plot_fcpm_intensities(I_fcpm, z_idx=z_mid)
fig.suptitle('Raw FCPM Intensities', y=1.02)
plt.tight_layout()
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_42507/2254853633.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Step 2: Preprocessing

In [7]:
# Normalize intensities
I_fcpm_norm = fcpm.normalize_fcpm(I_fcpm, method='global')

# Optional: Apply Gaussian smoothing to reduce noise
I_fcpm_smooth = fcpm.gaussian_filter_fcpm(I_fcpm_norm, sigma=0.5)

# Optional: Crop to region of interest
# I_fcpm_crop = fcpm.crop_fcpm_center(I_fcpm_smooth, size=(32, 32, 16))

print("Preprocessing complete:")
print(f"  Normalized: range [0, 1]")
print(f"  Smoothed: sigma=0.5")

Preprocessing complete:
  Normalized: range [0, 1]
  Smoothed: sigma=0.5


In [8]:
# Compare before/after preprocessing
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

angle = list(I_fcpm.keys())[0]
axes[0].imshow(I_fcpm[angle][:, :, z_mid], cmap='gray')
axes[0].set_title(f'Raw (angle={np.degrees(angle):.0f} deg)')
axes[0].axis('off')

axes[1].imshow(I_fcpm_smooth[angle][:, :, z_mid], cmap='gray')
axes[1].set_title('After preprocessing')
axes[1].axis('off')

plt.tight_layout()
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_42507/1108343570.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Step 3: Reconstruct Director Field

In [9]:
# Reconstruct using Q-tensor method
director_recon, info = fcpm.reconstruct(
    I_fcpm_smooth,
    fix_signs=True,   # Resolve nematic sign ambiguity
    verbose=True
)

print(f"\nReconstructed director shape: {director_recon.shape}")

Q-tensor reconstruction complete
Combined Sign Optimization

Phase 1: Chain propagation...
  After chain propagation: energy = 57073.92

Phase 2: Local flip refinement...
Initial gradient energy: 57073.92
  Iter 0: flipped 0 voxels, energy = 57073.92
Converged at iteration 0

Reconstructed director shape: (48, 48, 24)


### Step 4: Quality Assessment (No Ground Truth)

Without ground truth, we assess quality using **self-consistency metrics**:
- **Intensity RMSE**: How well does the reconstructed director reproduce the input FCPM?
- **Gradient Energy**: Smoothness of the reconstructed field (lower = smoother)

In [10]:
# Simulate FCPM from reconstructed director
I_fcpm_recon = fcpm.simulate_fcpm(director_recon)

# Compare with input
intensity_err = fcpm.intensity_reconstruction_error(I_fcpm_smooth, I_fcpm_recon)

print("=" * 50)
print("Self-Consistency Metrics (no ground truth)")
print("=" * 50)
print(f"Intensity RMSE (mean):     {intensity_err['rmse_mean']:.4f}")
print(f"Intensity RMSE (max):      {intensity_err['rmse_max']:.4f}")
print(f"Relative error:            {intensity_err['relative_error_mean']:.2%}")
print(f"Gradient energy:           {fcpm.reconstruction.gradient_energy(director_recon):.2f}")

Self-Consistency Metrics (no ground truth)
Intensity RMSE (mean):     0.1543
Intensity RMSE (max):      0.2134
Relative error:            18.67%
Gradient energy:           57073.92


In [11]:
# Compare input vs reconstructed FCPM intensities
fig, axes = plt.subplots(2, 4, figsize=(14, 7))

for i, angle in enumerate(I_fcpm_smooth.keys()):
    axes[0, i].imshow(I_fcpm_smooth[angle][:, :, z_mid], cmap='hot', vmin=0, vmax=1)
    axes[0, i].set_title(f'Input {np.degrees(angle):.0f} deg')
    axes[0, i].axis('off')
    
    axes[1, i].imshow(I_fcpm_recon[angle][:, :, z_mid], cmap='hot', vmin=0, vmax=1)
    axes[1, i].set_title(f'Reconstructed')
    axes[1, i].axis('off')

axes[0, 0].set_ylabel('Input', fontsize=12)
axes[1, 0].set_ylabel('Reconstructed', fontsize=12)
plt.suptitle('FCPM Intensity: Input vs Reconstructed', fontsize=14)
plt.tight_layout()
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_42507/1218750807.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Step 5: Visualize Reconstructed Director

In [12]:
# Director field slice
fig, ax = plt.subplots(figsize=(10, 10))
fcpm.plot_director_slice(director_recon, z_idx=z_mid, step=2, ax=ax, 
                        title=f'Reconstructed Director (z={z_mid})')
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_42507/1404026220.py:5: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [13]:
# Multiple z-slices
n_slices = 4
z_indices = np.linspace(0, director_recon.shape[2]-1, n_slices, dtype=int)

fig, axes = plt.subplots(1, n_slices, figsize=(16, 4))

for i, z_idx in enumerate(z_indices):
    fcpm.plot_director_slice(director_recon, z_idx=z_idx, step=2, ax=axes[i],
                            title=f'z = {z_idx}')

plt.suptitle('Director Field at Different Depths', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_42507/784247099.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [14]:
# nz component (out-of-plane)
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

im0 = axes[0].imshow(director_recon.nx[:, :, z_mid], cmap='RdBu_r', vmin=-1, vmax=1)
axes[0].set_title('nx (in-plane, x)')
axes[0].axis('off')
plt.colorbar(im0, ax=axes[0], fraction=0.046)

im1 = axes[1].imshow(director_recon.ny[:, :, z_mid], cmap='RdBu_r', vmin=-1, vmax=1)
axes[1].set_title('ny (in-plane, y)')
axes[1].axis('off')
plt.colorbar(im1, ax=axes[1], fraction=0.046)

im2 = axes[2].imshow(director_recon.nz[:, :, z_mid], cmap='RdBu_r', vmin=-1, vmax=1)
axes[2].set_title('nz (out-of-plane)')
axes[2].axis('off')
plt.colorbar(im2, ax=axes[2], fraction=0.046)

plt.suptitle(f'Director Components (z={z_mid})', fontsize=14)
plt.tight_layout()
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_42507/4120261459.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Advanced: Parameter Exploration

Try different preprocessing to improve reconstruction:

In [15]:
# Compare different filter strengths
sigmas = [0, 0.5, 1.0, 2.0]

fig, axes = plt.subplots(2, len(sigmas), figsize=(14, 7))

for i, sigma in enumerate(sigmas):
    # Apply filter
    if sigma > 0:
        I_filtered = fcpm.gaussian_filter_fcpm(I_fcpm_norm, sigma=sigma)
    else:
        I_filtered = I_fcpm_norm
    
    # Reconstruct
    d_recon, _ = fcpm.reconstruct(I_filtered, fix_signs=True, verbose=False)
    
    # Compute metrics
    I_recon = fcpm.simulate_fcpm(d_recon)
    err = fcpm.intensity_reconstruction_error(I_filtered, I_recon)
    grad_e = fcpm.reconstruction.gradient_energy(d_recon)
    
    # Plot FCPM
    angle = list(I_filtered.keys())[0]
    axes[0, i].imshow(I_filtered[angle][:, :, z_mid], cmap='gray')
    axes[0, i].set_title(f'sigma={sigma}')
    axes[0, i].axis('off')
    
    # Plot director
    fcpm.plot_director_slice(d_recon, z_idx=z_mid, step=3, ax=axes[1, i], title='')
    axes[1, i].set_xlabel(f'RMSE: {err["rmse_mean"]:.3f}\nEnergy: {grad_e:.0f}')

axes[0, 0].set_ylabel('Filtered FCPM', fontsize=11)
axes[1, 0].set_ylabel('Reconstructed', fontsize=11)
plt.suptitle('Effect of Gaussian Smoothing on Reconstruction', fontsize=14)
plt.tight_layout()
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_42507/2986973329.py:35: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Save Results

In [16]:
output_dir = Path('../output/experimental_reconstruction')
output_dir.mkdir(parents=True, exist_ok=True)

# Save reconstructed director
fcpm.save_director_npz(director_recon, output_dir / 'director_reconstructed.npz')

# Export for other software
fcpm.export_for_vtk(director_recon, output_dir / 'director.vtk')

print(f"Results saved to: {output_dir.absolute()}")
print("\nFiles:")
for f in output_dir.iterdir():
    print(f"  - {f.name}")

Results saved to: /Users/narekmeloyan/PycharmProjects/FCPM-Simulation/examples/../output/experimental_reconstruction

Files:
  - director.vtk
  - director_reconstructed.npz


## Loading Different File Formats

Examples of how to load FCPM data from various sources:

In [17]:
# From NPZ file (auto-detection)
# I_fcpm = fcpm.load_fcpm('data.npz')

# From TIFF stack directory (requires angles)
# I_fcpm = fcpm.load_fcpm_tiff_stack(
#     'tiff_folder/',
#     angles=[0, np.pi/4, np.pi/2, 3*np.pi/4]
# )

# From MATLAB .mat file
# I_fcpm = fcpm.load_fcpm_mat('data.mat')

# From NumPy arrays directly
# I_fcpm = {
#     0.0: np.load('intensity_0deg.npy'),
#     np.pi/4: np.load('intensity_45deg.npy'),
#     np.pi/2: np.load('intensity_90deg.npy'),
#     3*np.pi/4: np.load('intensity_135deg.npy'),
# }

print("See examples above for loading different file formats.")

See examples above for loading different file formats.
